In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # LLM이 우리가 원하는 형태로 응답을 하지 않으면 에러를 고쳐보려고 하도록 함
    handle_parsing_errors=True,
    tools=[
        # Zero-shot은 StructuredTool이 아닌 Tool을 이용
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to find the sum of all these numbers.
Action: Sum Calculator
Action Input: 355.39,924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 1280.26,721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 2001.46,1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 3941.75,573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4515.38,65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4581.1,35.00
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4616.1,552.00
Observation: 5168.1
Thou

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}